In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

In [2]:
pwd

'C:\\Users\\ygy91\\Desktop\\GX sales report'

In [3]:
# import data
histo_folder = ['2020 sales']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

temp = []
for i in month:
    temp.append(pd.read_csv(f'2020 sales//{i}.csv'))
df = pd.concat(temp).reset_index(drop = True)

In [4]:
cate = pd.read_excel('sup source//category.xlsx', sheet_name = 'Category')

In [5]:
# drop unneeded columns
col_drop = ['Warehouse Name', 'Must Ship By', 'Backorder Date', 'Shipping Account Number', 'Ship To Address', 'Ship To Address 2',
            'Ship To Zip', 'Ship To Phone', 'Inventory Send Date', 'Registered Timestamp', 'Customization Text', 'Event Name',
            'Event ID', 'Event Start Date', 'Event End Date', 'Event Type', 'Backorder Reason', 'Original Product ID', 
            'Original Product Name', 'Event Inventory Source', 'Packing Slip URL', 'Tracking Number', 'Ready for Pickup Date',
            'Destination Country', 'Depot ID', 'Depot Name', 'Wholesale Event Source', 'Wholesale Event Store Source', 
            'Composite Wood Product', 'Sales Channel']

In [6]:
df.drop(col_drop, axis = 1, inplace = True)

In [7]:
# Date Cleaning
date_col = ['PO Date', 'Must Ship By', 'Backorder Date']

df['PO Date'] = df.apply(lambda x: datetime.strptime(x['PO Date'],'%m/%d/%Y'), axis = 1)
df['Month'] = df.apply(lambda x: x['PO Date'].month, axis = 1)

In [8]:
# add category
df = df.merge(cate[['Item Name', 'Category']], on = 'Item Name', how = 'left')

In [9]:
# clean Item Number
df['Item Number'] = df.apply(lambda x: x['Item Number'][2:-1], axis = 1)

In [10]:
# clean PO Date & Time
df['PO Time'] = df['PO Date & Time'].str[11:13]

---

In [11]:
# process product info

df_pi = pd.read_csv('sup source//product info.csv')

In [12]:
df_pi['review & rating'] = df_pi.apply(lambda x: str(x['rating']).strip(str(x['Item Name'])), axis = 1)

In [13]:
p = re.compile(r'[(](.*?)[)]', re.S)

In [14]:
df_pi['Review'] = df_pi.apply(lambda x: 0
                        if re.findall(p, x['review & rating']) == [] 
                        else re.findall(p, x['review & rating'])[0], axis =1 )

In [15]:
df_pi['Rating'] = df_pi.apply(lambda x: x['review & rating'][x['review & rating'].index(':')+1:] if ':' in x['review & rating'] else 0, axis = 1)

In [16]:
df_pi['Date Launched'] = df_pi.apply(lambda x: datetime.strptime(x['Date Launched'],'%m/%d/%Y'), axis = 1)

In [17]:
df_pi = df_pi[['SKU','Date Launched','Satus','Review','Rating']].drop_duplicates()

In [18]:
df_pi

,SKU,Date Launched,Satus,Review,Rating
0,TNFI2298,2020-11-30,Live,0,0
1,TNFI2230,2020-10-02,Live,0,0
2,TNFI2240,2020-10-02,Live,0,0
3,TNFI2236,2020-10-02,Live,0,0
4,TNFI2235,2020-10-02,Live,0,0
...,...,...,...,...,...
790,TNFI1014,2018-03-28,Live,9,4.4
791,HAZE2464,2017-09-11,Live,8,4.9
792,HAZE2467,2017-09-11,Live,445,4.2
793,HAZE2468,2017-09-11,Live,1508,4.8


In [19]:
df=df.merge(df_pi, on ='SKU',how = 'left')

---

## Inventory

In [20]:
pwd

'C:\\Users\\ygy91\\Desktop\\GX sales report'

In [21]:
df_inv = pd.read_excel('inventory//20201225全球库存统计及补货计划表.xlsx', sheet_name = '入库商家补货计划表')

In [22]:
# 1. good columns

cols = ['国家', '对应账号SKU', '类别category',   '销量趋势(前30天对比（前60天到前30天）销量)',
        '第一次断货时间', '第二次断货时间', '要求ETD', '要求验货时间',
        '要求完工时间\n订舱\n调拨单', '要求下单时间', '建议下单数量', '货号', '入库商家',
       '机构', '四字机构', 'CBM', 'FOB出运港', '工厂']

warehouses = ['Cloud Stock\n(2019年04月01日至今)',
               'ComingSoon\n(2019年04月01日至今)', 'Ship Now\n(2019年04月01日至今)',
               'On the Way',
               'On the Way 2\n(Wayfair-Castle Gate仓\n US-Castle Gate仓\n CA - Castle Gate仓\n WAYFAIR-DE\n WAYFAIR-FR\n WAYFAIR-UK)',
               '仓1', '仓2', '仓3', 'Castle Gate仓', 'FBA仓',
               'FBC仓\n(对于MX，此列为FBM仓【Mercado Libre平台库存】)',
               '出口易库存仓1\n(对于US 美国，仓1为【美国库存】,\n对于CA 加拿大，仓1为【加拿大库存】)\n对于EU 欧洲，仓1为【英国库存】)',
               '出口易库存仓2\n(对于EU 欧洲，仓2为【法国库存】)', '出口易库存仓3\n(对于EU 欧洲，仓3为【德国库存】)', 'K2',
               '天池(FR LVA)', '兼爱(FR LBG)', '说剑(US SNA)', '美东一海通（天下）\n(US ATL)',
               '美东（天运）\n(US PHL)', '美西一海通（达生）\n(US BUR)', '美西（天道）\n(US LSQ)',
               '圆融汇通英国(让王)\n(UK MAN)', '圆融汇通（逍遥）\n(CA YTZ)', '德国嘉宏（山木）\n(DE DUS)',
               '大森林（宗师）\n(FR BVA)', 'Castle\n(OVERSTOCK)', 'CPA\n(OVERSTOCK)',
               'WKC\n(OVERSTOCK)', 'US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK)',
               '公冶(FR BOD)', 'Groupon仓\n(NL AMS)']

In [23]:
df_unpivot = df_inv.melt(id_vars = cols, value_vars = warehouses, var_name = 'warehouses')

In [24]:
map_WH  =   {'1. WH-Cloud': ['Cloud Stock\n(2019年04月01日至今)'],
             '2. WH-Coming': ["ComingSoon\n(2019年04月01日至今)"],
             '3. WH-ReadyToShip': ['Ship Now\n(2019年04月01日至今)'],
             '4. On the Way': ['On the Way', 'On the Way 2\n(Wayfair-Castle Gate仓\n US-Castle Gate仓\n CA - Castle Gate仓\n WAYFAIR-DE\n WAYFAIR-FR\n WAYFAIR-UK)'],
             '5. WH-Arrival': ['仓1', '仓2', '仓3', 'Castle Gate仓', 'FBA仓',
                     'FBC仓\n(对于MX，此列为FBM仓【Mercado Libre平台库存】)',
                     '出口易库存仓1\n(对于US 美国，仓1为【美国库存】,\n对于CA 加拿大，仓1为【加拿大库存】)\n对于EU 欧洲，仓1为【英国库存】)',
                     '出口易库存仓2\n(对于EU 欧洲，仓2为【法国库存】)', '出口易库存仓3\n(对于EU 欧洲，仓3为【德国库存】)',
                     'K2', '天池(FR LVA)', '兼爱(FR LBG)', '说剑(US SNA)',
                     '美东一海通（天下）\n(US ATL)', '美东（天运）\n(US PHL)', '美西一海通（达生）\n(US BUR)',
                     '美西（天道）\n(US LSQ)', '圆融汇通英国(让王)\n(UK MAN)', '圆融汇通（逍遥）\n(CA YTZ)',
                     '德国嘉宏（山木）\n(DE DUS)', '大森林（宗师）\n(FR BVA)', 'Castle\n(OVERSTOCK)',
                     'CPA\n(OVERSTOCK)', 'WKC\n(OVERSTOCK)',
                     'US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK)', '公冶(FR BOD)',
                     'Groupon仓\n(NL AMS)']}

In [25]:
# select columns
df_unpivot.columns

Index(['国家', '对应账号SKU', '类别category', '销量趋势(前30天对比（前60天到前30天）销量)', '第一次断货时间',
       '第二次断货时间', '要求ETD', '要求验货时间', '要求完工时间\n订舱\n调拨单', '要求下单时间', '建议下单数量',
       '货号', '入库商家', '机构', '四字机构', 'CBM', 'FOB出运港', '工厂', 'warehouses',
       'value'],
      dtype='object')

In [26]:
df_wh = df_unpivot[['国家', '对应账号SKU', '类别category', '机构', '四字机构', 'CBM', 'FOB出运港', '工厂', 'warehouses', 'value']]

df_wh.columns = ['Country', 'Item Number', 'Category', 'Operation detail', 'Department-4', 
              'CBM', 'FOB-City', 'Factory', 'warehouse detail', 'InvBal-Qty']

df_wh = df_wh[df_wh['InvBal-Qty'].notnull()].reset_index(drop=True)

In [27]:
map_WH.keys()

dict_keys(['1. WH-Cloud', '2. WH-Coming', '3. WH-ReadyToShip', '4. On the Way', '5. WH-Arrival'])

In [28]:
maps = []

for i in map_WH:
    temp = pd.DataFrame(map_WH[i], columns=['warehouse detail'])
    temp['Warehouse'] = i
    maps.append(temp)

maps_wh = pd.concat(maps).reset_index(drop = True)


In [29]:
maps_wh.head()

,warehouse detail,Warehouse
0,Cloud Stock\n(2019年04月01日至今),1. WH-Cloud
1,ComingSoon\n(2019年04月01日至今),2. WH-Coming
2,Ship Now\n(2019年04月01日至今),3. WH-ReadyToShip
3,On the Way,4. On the Way
4,On the Way 2\n(Wayfair-Castle Gate仓\n US-Castl...,4. On the Way


In [30]:
df_wh = df_wh.merge(maps_wh, on = 'warehouse detail', how = 'left')

In [31]:
################################   select only THTJ inventory   #################################

In [32]:
# df_wh.to_csv('PowerBI Input//Inventory Balances.csv', index=False)

---

## Product Catalog

In [33]:
# sales items
df_sales_item = df[['Item Number','SKU']]

# Upper
df_sales_item['Item Number'] = df_sales_item['Item Number'].str.upper()
df_sales_item['SKU'] = df_sales_item['SKU'].str.upper()
# clean '?' and strip
df_sales_item['Item Number'] = df_sales_item.apply(lambda x: str(x['Item Number']).replace('?',' ').strip(), axis =1)

df_sales_item = df_sales_item.drop_duplicates().reset_index(drop=True)

In [34]:
df_prod = pd.read_csv('sup source//Product Catalog//Product Catalog downloaded from Wayfair - 20201230.csv')

In [35]:
# product catalog items
df_rel = df_prod[['Supplier Part #', 'Internal SKU', 'Product Name']]

#Upper
df_rel['Supplier Part #'] = df_rel['Supplier Part #'].str.upper()
df_rel['Internal SKU'] = df_rel['Internal SKU'].str.upper()
df_rel['Product Name'] = df_rel['Product Name'].str.upper()
# clean '?' and strip
df_rel['Supplier Part #'] = df_rel.apply(lambda x: str(x['Supplier Part #']).replace('?',' ').strip(), axis =1)

df_rel = df_rel.drop_duplicates()
df_rel.rename(columns={'Supplier Part #':'Item Number', 'Internal SKU':'SKU'}, inplace=True)
df_rel = df_rel[df_rel['SKU'].notnull()].reset_index(drop=True)

In [36]:
df_item = pd.concat([df_rel[['Item Number', 'SKU']],df_sales_item]).drop_duplicates()

In [37]:
df_rel_name = df_rel[['SKU','Product Name']].drop_duplicates()

In [38]:
df_item = df_item.merge(df_rel_name, on='SKU', how='left')

In [39]:
# groupby Item Number ==> Make Sure item number are unique
df_item.sort_values(by = 'SKU', inplace = True)

In [40]:
df_item['SKU'] = ' && ' + df_item['SKU']
df_item['Product Name'] = ' && ' + df_item['Product Name']

In [41]:
df_item['SKU'] = df_item['SKU'].astype(str)
df_item['Product Name'] = df_item['Product Name'].astype(str)

In [42]:
df_item = df_item.groupby('Item Number').sum().reset_index()

In [43]:
df_item['SKU'] = df_item['SKU'].str[4:]
df_item['Product Name'] = df_item['Product Name'].str[4:]

In [44]:
df_item.fillna('Product Name not found in PC',inplace = True)

In [45]:
# finish processing sales and product catalog

In [46]:
# warehouse items
df_wh_item = df_wh[['Item Number']]

# upper
df_wh_item['Item Number'] =  df_wh_item['Item Number'].str.upper()
# clean '?' and strip
df_wh_item['Item Number'] = df_wh_item.apply(lambda x: str(x['Item Number']).replace('?',' ').strip(), axis =1)

df_wh_item = df_wh_item.drop_duplicates().reset_index(drop=True)

In [47]:
df_wh_item['in/out'] = df_wh_item.apply(lambda x: x['Item Number'] in list(df_item['Item Number']), axis = 1)

In [48]:
df_add = df_wh_item[~df_wh_item['in/out']][['Item Number']]
df_add['SKU'] = 'unallocated'      # define unallocated items

In [49]:
df_itemset = pd.concat([df_item, df_add]).fillna('unallocated inventory')    

In [50]:
df_itemset.to_csv('Item Mapping//DIM Item Name.csv', index=False, encoding='utf_8_sig')

In [51]:
pwd

'C:\\Users\\ygy91\\Desktop\\GX sales report'

In [52]:
df_prod = pd.read_csv('sup source//Product Catalog//Product Catalog downloaded from Wayfair - 20201230.csv')

In [53]:
# create DIM-SKU from Product Catalog downloaded
df_ProductDIM = df_prod[['Internal SKU','Product Name']].drop_duplicates().dropna(axis=0).sort_values(by = 'Internal SKU')

In [54]:
# to include missing items in df_ProductDIM (downloaded from FUll Product Catalog but still missing in sales, weird)
Sales_Additional_Product_DIM = df[['SKU']].drop_duplicates()

In [55]:
df_ProductDIM.rename(columns = {'Internal SKU': 'SKU'}, inplace = True)

In [56]:
df_ProductDIM = df_ProductDIM.merge(Sales_Additional_Product_DIM, on = 'SKU', how = 'outer')

In [57]:
# 
temp = df_prod.copy()
temp = temp.rename(columns = {'Supplier Part #':'Item Number'})
# upper: make sure match on Item Number
temp['Item Number'] = temp['Item Number'].str.upper()

In [58]:
# create bridge between Inventory and Sales
df_wh_after_add_internalSKU = df_wh.merge(temp[['Item Number', 'Internal SKU']].drop_duplicates(), on = 'Item Number', how = 'left')

In [59]:
DIM_Item_Name = df_wh_after_add_internalSKU[['Item Number', 'Internal SKU']].fillna('Unallocated').drop_duplicates()
DIM_Item_Name.rename(columns= {'Internal SKU':'SKU'}, inplace = True)

In [60]:
DIM_Item_Name = DIM_Item_Name.merge(df_ProductDIM, on = 'SKU', how = 'left')

In [61]:
DIM_Item_Name['Product Name'] = '&' + DIM_Item_Name['Product Name']
DIM_Item_Name['SKU'] = '&' + DIM_Item_Name['SKU']

In [62]:
DIM_Item_Name = DIM_Item_Name.sort_values(by = 'SKU').groupby(['Item Number']).sum().reset_index()

DIM_Item_Name['SKU'] = DIM_Item_Name['SKU'].str[1:]
DIM_Item_Name['Product Name'] = DIM_Item_Name['Product Name'].str[1:]

In [63]:
DIM_Item_Name[['SKU']].drop_duplicates()

,SKU
0,TNFI1964
1,TNFI1965
2,TNFI1335
3,Unallocated
5,TNFI1289
...,...
4399,TNFI1285
4405,TNFI2103
4413,TNFI1633
4421,TNFI1923


In [64]:
DIM_Item_Name.to_csv('Item Mapping//DIM Item Name.csv', index=False, encoding='utf_8_sig')

In [65]:
DIM_Item_Name

,Item Number,SKU,Product Name
0,ABELARDO,TNFI1964,Ajaye Armoire
1,ABELARDO THREE DOORS,TNFI1965,Ajaye Armoire
2,ABLITT,TNFI1335,Hawkinson Labounty Executive Chair
3,ABLOTT BLACK,Unallocated,NaN
4,ABLOTT BROWN,Unallocated,NaN
...,...,...,...
4499,ZOMBA YELLOW FELLCE,Unallocated,NaN
4500,ZOMBALUN DARK BLUE Ⅰ,Unallocated,NaN
4501,ZOMBALUN GREY Ⅰ,Unallocated,NaN
4502,ZURICH,Unallocated,NaN


---

## Now save Inventory Fact Table (after finishing item mapping)

In [66]:
df_wh = df_wh.merge(DIM_Item_Name, on = 'Item Number', how = 'left')

In [67]:
df_wh.to_csv('PowerBI Input//Inventory Balances.csv', index=False)

## Now save Sales Fact Table (after finishing item mapping)

In [68]:
df.rename(columns={'SKU':'Wayfair SKU'}, inplace = True)

In [69]:
df = df.merge(DIM_Item_Name, on = 'Item Number', how = 'left')

In [70]:
df.to_csv('PowerBI Input//wayfair_sales_input.csv', index = False)

In [71]:
DIM_Item_Name

,Item Number,SKU,Product Name
0,ABELARDO,TNFI1964,Ajaye Armoire
1,ABELARDO THREE DOORS,TNFI1965,Ajaye Armoire
2,ABLITT,TNFI1335,Hawkinson Labounty Executive Chair
3,ABLOTT BLACK,Unallocated,NaN
4,ABLOTT BROWN,Unallocated,NaN
...,...,...,...
4499,ZOMBA YELLOW FELLCE,Unallocated,NaN
4500,ZOMBALUN DARK BLUE Ⅰ,Unallocated,NaN
4501,ZOMBALUN GREY Ⅰ,Unallocated,NaN
4502,ZURICH,Unallocated,NaN


In [72]:
missing = df_wh_after_add_internalSKU[(df_wh_after_add_internalSKU['Country'] == 'US') | (df_wh_after_add_internalSKU['Country'] == 'CA')]

missing = missing[missing['Internal SKU'].isnull()]
missing

,Country,Item Number,Category,Operation detail,Department-4,CBM,FOB-City,Factory,warehouse detail,InvBal-Qty,Warehouse,Internal SKU
2116,CA,ADAMS BLACK,Secretary Office Chair,青荇渡_39FINC,太华天街,0.135,宁波,安吉鼎悦家具有限公司,Cloud Stock\n(2019年04月01日至今),50.0,1. WH-Cloud,NaN
2117,CA,ADAMS BROWN,Secretary Office Chair,曾厝垵,太华天街,0.135,上海,安吉龙派家具有限公司,Cloud Stock\n(2019年04月01日至今),200.0,1. WH-Cloud,NaN
2118,CA,ADAMS BROWN,Secretary Office Chair,青荇渡_39FINC,太华天街,0.135,上海,安吉龙派家具有限公司,Cloud Stock\n(2019年04月01日至今),50.0,1. WH-Cloud,NaN
2120,CA,ALASKA WOOD,Board with Metal Desk,名古屋_GX,太华天街,0.105,厦门,漳州红梅家具有限公司,Cloud Stock\n(2019年04月01日至今),300.0,1. WH-Cloud,NaN
2121,CA,ALBERTIN,L-Shaped & Corner Desk,曾厝垵_39FINC,太华天街,0.115,厦门,新佳美(漳州)日用品有限公司,Cloud Stock\n(2019年04月01日至今),300.0,1. WH-Cloud,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24407,US,ROOKIE ROUND TABLE GREY,Dining Table,奈良井,太华天街,0.114,天津,廊坊赛普贸易有限公司,US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK),17.0,5. WH-Arrival,NaN
24423,US,SCARGILL DARK GREY Ⅰ,Dining Chair,奈良井,太华天街,0.200,天津,廊坊森雅家具有限公司,US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK),88.0,5. WH-Arrival,NaN
24427,US,SCARGILL YELLOW,Dining Chair,奈良井,太华天街,0.184,天津,廊坊森雅家具有限公司,US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK),46.0,5. WH-Arrival,NaN
24428,US,SMEG ROSE 32MM A,Dining Chair,奈良井,太华天街,0.139,天津,廊坊森雅家具有限公司,US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK),46.0,5. WH-Arrival,NaN
